# Setup

In [ ]:
from ultralytics import YOLO
from matplotlib import pyplot as plt
import cv2
import torch
import os
import numpy as np
import random
import json
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = "0, 1, 2, 3" # replace with the GPU IDs that are available
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['PYTORCH_NO_CUDA_MEMORY_CACHING'] = "1"

# to help with reproducibility (inspired from YOLO init_seeds function)
seed=0
import torch.backends.cudnn as cudnn 
random.seed(seed) 
np.random.seed(seed) 
torch.manual_seed(seed) 
cudnn.benchmark, cudnn.deterministic = (False, True) if seed == 0 else (True, False) 

PYTORCH_NO_CUDA_MEMORY_CACHING=1
np.set_printoptions(precision=9)

sys.path.append('../') # workaround for lack of relative import in notebooks
from sam_predictor import predictor_utils, astro_sam #, residualAttentionBlock
from losses import metrics_utils
from dataset import dataset_utils, coco_to_yolo_converter

In [ ]:
kfold_iter=0

device_id=3
batch_size=8
lr=3e-5
wd=0.0005
wandb_track=True
# torch.cuda.set_device(device_id)

if wandb_track:
    from datetime import datetime
    # !pip install wandb
    # !wandb login --relogin
    import wandb
    wandb.login()
    run = wandb.init(project="yolo-sam", name=f"yolo-sam {datetime.now()}")

## Dataset

In [ ]:
convert = True # if True, convert coco to yolo format

if convert: 
    coco_to_yolo_converter.convert_coco_to_yolo(
    dir_absolute_path='/workspace/raid/test/XAMI/',
    dataset_path='../xami_dataset/',
    yolo_dataset_path='../xami_dataset_YOLO/'
)
else:
    yolo_dataset_path = f'../xami_dataset_YOLO/' # replace with the path to the YOLO dataset

In [ ]:
import yaml

with open(yolo_dataset_path+f"data.yaml", 'r') as stream:
    yam_data = yaml.safe_load(stream) # dictionary with keys 'names', 'nc', 'train', 'val'

classes = {i:name for i, name in enumerate(yam_data['names'])}
train_path = yam_data['train']
val_path = yam_data['val']
print(classes)

coco_data_path = f'../../XAMI-dataset/notebooks/mskf_{kfold_iter}/'
annotations_file = '_annotations.coco.json'

with open(coco_data_path+'train/'+annotations_file, 'r') as f1, open(coco_data_path+'valid/'+annotations_file, 'r') as f2:
    train_coco_data = json.load(f1)
    valid_coco_data = json.load(f2)

In [ ]:
train_dir = yolo_dataset_path+f'train/images/'
valid_dir = yolo_dataset_path+f'valid/images/'

train_image_files = os.listdir(train_dir)
valid_image_files = os.listdir(valid_dir)

# Model setup

In [ ]:
import sys
sys.path.append(sys.path[0]+'/../mobile_sam/')
from mobile_sam import sam_model_registry, SamPredictor

device = f"cuda:{device_id}" if torch.cuda.is_available() else "cpu"
mobile_sam_checkpoint = '../train/weights/sam_weights/sam_0_best.pth'
# mobile_sam_checkpoint = '../train/weights/yolo_sam_final.pth'

yolov8_pretrained_model = YOLO(f'../train/weights/yolo_weights/best.pt');
yolov8_pretrained_model.to(f'cuda:{device_id}');
mobile_sam_model = sam_model_registry["vit_t"](checkpoint=mobile_sam_checkpoint)
mobile_sam_model.to(device)
predictor = SamPredictor(mobile_sam_model)

# # Residual Attention Block
# residual_block = residualAttentionBlock.ResidualAttentionBlock(d_model=256, n_head=8, mlp_ratio=4.0).to(device)
# residual_block.load_state_dict(torch.load('../output_sam/residual_attn_blk_2024-05-10 08:05:54.498084_best.pth'))

# for name, param in residual_block.named_parameters():
# 	param.requires_grad = True 

In [ ]:
astrosam_model = astro_sam.AstroSAM(mobile_sam_model, device, predictor)

In [ ]:
for name, param in mobile_sam_model.named_parameters():
    params_to_train = ['mask_tokens', 'output_upscaling', 'output_hypernetworks_mlps', 'iou_prediction_head']
    if 'mask_decoder' in name: # and any(s in name for s in params_to_train):
        param.requires_grad = True
    else:
        param.requires_grad = False
        
print(f"🚀 The model has {sum(p.numel() for p in astrosam_model.model.parameters() if p.requires_grad)} trainable parameters.")
# print(f"🚀 The residual attention block has {sum(p.numel() for p in astrosam_model.residualAttentionBlock.parameters() if p.requires_grad)} trainable parameters.\n")

# predictor_utils.check_requires_grad(astrosam_model.model)
# predictor_utils.check_requires_grad(astrosam_model.residualAttentionBlock)

In [ ]:
import time
import torch.nn.functional as F
import tqdm
from tqdm import tqdm

train_num_batches = len(train_image_files) // batch_size
valid_num_batches = len(valid_image_files) // batch_size
parameters_to_optimize = [param for param in mobile_sam_model.mask_decoder.parameters() if param.requires_grad]
optimizer = torch.optim.AdamW(parameters_to_optimize, lr=lr, weight_decay=wd) if len(parameters_to_optimize) > 0 else None

## Define metrics

In [ ]:
from torch import tensor
from torchmetrics.detection import MeanAveragePrecision
from pprint import pprint

metric_thresholds = [[0.5], [0.75], [0.5, 0.9]]

# Train

In [ ]:
from collections import defaultdict

best_valid_loss = float('inf')
num_epochs = 10 
n_epochs_stop = 100 # 5+ num_epochs//10
all_metrics = defaultdict(dict)
compute_metrics = True

In [ ]:
# Intro
predictor_utils.print_training_intro(
    train_image_files, valid_image_files, device, metric_thresholds, num_epochs, 
    batch_size, lr, wd, wandb_track, mobile_sam_model, 'AdamW')

# Training loop
for epoch in range(num_epochs):

    # Train
    astrosam_model.model.train()
    # astrosam_model.model.eval()
    
    # astrosam_model.residualAttentionBlock.train()
    train_results = astrosam_model.run_yolo_sam_epoch(
        yolov8_pretrained_model,
        phase='train',
        batch_size=batch_size, 
        image_files=train_image_files, 
        images_dir=train_dir, 
        num_batches=train_num_batches,
        optimizer=optimizer) 
    
    epoch_sam_loss_train, train_preds, train_gts = train_results[:3]
    train_gt_classes, train_pred_classes, train_all_iou_scores, train_mask_areas, _ = train_results[3:]

    # Validate
    astrosam_model.model.eval()
    # astrosam_model.residualAttentionBlock.eval()
    
    with torch.no_grad():
        valid_results = astrosam_model.run_yolo_sam_epoch(
            yolov8_pretrained_model,
            phase='val',
            batch_size=batch_size, 
            image_files=valid_image_files, 
            images_dir=valid_dir, 
            num_batches=valid_num_batches,
            optimizer=None) 

    epoch_sam_loss_val, valid_preds, valid_gts = valid_results[:3]
    valid_gt_classes, valid_pred_classes, valid_all_iou_scores, valid_mask_areas, pred_images = valid_results[3:]

    # Early stopping                                                                                
    if epoch_sam_loss_val < best_valid_loss:
        best_valid_loss = epoch_sam_loss_val
        best_model = astrosam_model.model
        best_epoch = epoch
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve == n_epochs_stop:
            print("Early stopping initiated.")
            early_stop = True
            break
        
    # Metrics
    if compute_metrics:
        for threshold in tqdm(metric_thresholds, desc="(Metrics) Processing thresholds", bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}'):
            metric = MeanAveragePrecision(
            iou_type = "segm", 
            iou_thresholds = threshold, 
            max_detection_thresholds=[1, 10, 100],
            class_metrics=True,
            extended_summary=False)
            
            train_metrics = metrics_utils.mAP_metrics(metric,
            train_preds,
            train_gts, 
            train_gt_classes, 
            train_pred_classes, 
            train_all_iou_scores,
            train_mask_areas,
            show_metrics=False) 
                
            valid_metrics = metrics_utils.mAP_metrics(metric,
                    valid_preds,
                    valid_gts, 
                    valid_gt_classes, 
                    valid_pred_classes, 
                    valid_all_iou_scores,
                    valid_mask_areas,
                    show_metrics=False)
        
            all_metrics[tuple(threshold)] = {'train': train_metrics, 'valid': valid_metrics}

        # Prints
        if not wandb_track:
            wandb = None
        predictor_utils.prints_and_wandb(
            epoch, 
            epoch_sam_loss_train, 
            epoch_sam_loss_val, 
            all_metrics, 
            metric_thresholds,
            wandb)

#     # Checkpoint save
#     torch.save(best_model.state_dict(), f'./output_sam/yolo_sam_{epoch}.pth')

# torch.save(best_model.state_dict(), f'./output_sam/yolo_sam_final.pth')

# Finish wandb run
if wandb_track:
    run.finish()

In [ ]:
# Epoch 12. Train loss: 0.390036
# Epoch 12. Validation loss: 0.3961745
# Train mAP50: tensor(0.9316) Train mAP75: tensor(0.7344) Train mAP50-90: tensor(0.6263)
# Valid mAP50: tensor(0.8694) Valid mAP75: tensor(0.6692) Valid mAP50-90: tensor(0.5737)

In [ ]:
# Epoch 16. Train loss: 0.3887573
# Epoch 16. Validation loss: 0.3952929
# Train mAP50: tensor(0.9238) Train mAP75: tensor(0.7429) Train mAP50-90: tensor(0.6288)
# Valid mAP50: tensor(0.8665) Valid mAP75: tensor(0.6799) Valid mAP50-90: tensor(0.5777)

In [ ]:
# Epoch 12. Train loss: 0.3895514
# Epoch 12. Validation loss: 0.3958057
# Train mAP50: tensor(0.9384) Train mAP75: tensor(0.7349) Train mAP50-90: tensor(0.6285)
# Valid mAP50: tensor(0.8759) Valid mAP75: tensor(0.6678) Valid mAP50-90: tensor(0.5741)

In [ ]:
best_epoch

In [ ]:
all_metrics 

# Metrics visualisation

In [ ]:
def ious_pred_vs_gt(gts, preds):
    all_ious_pred_vs_gt_flatten = []
    all_ious_pred_vs_gt = []
    
    for i in range(len(gts)):
        iou_image_scores = []
        for gt_mask, pred_mask in zip(gts[i], preds[i]):
            intersection = np.logical_and(gt_mask, pred_mask)
            union = np.logical_or(gt_mask, pred_mask)
            iou_score = np.sum(intersection) / np.sum(union) if np.sum(union) > 0 else 0
            iou_image_scores.append(iou_score)
            all_ious_pred_vs_gt_flatten.append(iou_score)
        all_ious_pred_vs_gt.append(iou_image_scores)
        
    return all_ious_pred_vs_gt_flatten, all_ious_pred_vs_gt

train_all_ious_pred_vs_gt, train_ious_pred_vs_gt = ious_pred_vs_gt(train_gts, train_preds)
valid_all_ious_pred_vs_gt, valid_ious_pred_vs_gt = ious_pred_vs_gt(valid_gts, valid_preds)

In [ ]:
# IoUs frequency

train_all_ious_flatten, train_mask_areas_flatten = metrics_utils.flatten_ious_areas(train_pred_classes, train_all_iou_scores, train_mask_areas)
valid_all_ious_flatten, valid_mask_areas_flatten = metrics_utils.flatten_ious_areas(valid_pred_classes, valid_all_iou_scores, valid_mask_areas)

In [ ]:
# Plotting the histogram
plt.figure(figsize=(10, 6))
plt.hist(valid_all_ious_pred_vs_gt, bins=30, alpha=0.7, color='blue', edgecolor='black')
plt.title('Segmentation Predicted IoUs')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
from matplotlib import rcParams

rcParams.update({'font.size': 14, 'axes.labelsize': 16, 'axes.titlesize': 18, 'xtick.labelsize': 12, 'ytick.labelsize': 12, 'legend.fontsize': 14, 'font.family':'sans-serif'})

ious_train = train_all_ious_pred_vs_gt
ious_valid = valid_all_ious_pred_vs_gt
areas_train = train_mask_areas_flatten
areas_valid = valid_mask_areas_flatten

vals_train, base_train = np.histogram(ious_train, bins=1000, range=(0, 1), density=True)
cumulative_train = np.cumsum(vals_train * np.diff(base_train))
vals_valid, base_valid = np.histogram(ious_valid, bins=1000, range=(0, 1), density=True)
cumulative_valid = np.cumsum(vals_valid * np.diff(base_valid))

plt.figure(figsize=(5, 5))
plt.plot(base_train[:-1], cumulative_train, label='Train', color='magenta')
plt.plot(base_valid[:-1], cumulative_valid, label='Validation', color='blue')

quartiles_y = np.quantile(cumulative_train, [0.8, 0.9])
quartiles_x = np.interp(quartiles_y, cumulative_train, base_train[:-1])

for qx, qy in zip(quartiles_x, quartiles_y):
    plt.hlines(qy, xmin=0, xmax=qx, colors='grey', linestyles='dashed')
    plt.vlines(qx, ymin=0, ymax=qy, colors='grey', linestyles='dashed')
    plt.text(qx, qy, f'({qx:.2f}, {qy:.2f})', fontsize=18, verticalalignment='bottom', horizontalalignment='right')

plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel('IoU', fontsize=22)
plt.ylabel('Cumulative Frequency', fontsize=22)
plt.legend(fontsize=18, loc='upper left')
plt.tight_layout()
plt.savefig('../plots/cdf_ious.png', dpi=500)
plt.show()

#D81B60
#1E88E5

#1A85FF
#D41159
plt.figure(figsize=(5, 5))
plt.scatter(np.log(areas_train), ious_train, alpha=0.5, label='Train', color='magenta')
plt.scatter(np.log(areas_valid), ious_valid, alpha=0.4, label='Validation', color='blue')
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
# x_ticks_positions = np.array([20000, 40000, 60000])*(np.max(np.log(areas_train))- np.min(np.log(areas_train)))/(np.max(areas_train)-np.min(areas_train))   # show orignal ticks but log plot
# x_ticks_labels = ['20000', '40000', '60000']

# plt.xticks(x_ticks_positions, x_ticks_labels, fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

plt.xlabel('Mask area [log(px)]', fontsize=22)
plt.ylabel('IoU', fontsize=22)
plt.legend(fontsize=18, loc='lower right', bbox_to_anchor=(1, 0.04))
plt.tight_layout()
plt.savefig('../plots/ious_and_areas_log.png', dpi=500)
plt.show()

In [ ]:
np.max(areas_train), np.max(areas_valid)

In [ ]:
np.min(ious_train), np.min(ious_valid)

In [ ]:
np.min(ious_train), np.max(ious_train), np.min(ious_valid), np.max(ious_valid)

### IoU TP FP FN plot (~29s)

In [ ]:
def compute_metrics_with_range(gt_masks, pred_masks, iou_thresholds, image=None):
    """Compute the True Positive, False Positive, and False Negative BINARY masks for multiple segmentations."""
    
    combined_gt_mask = np.zeros_like(gt_masks[0][0], dtype=bool)
    combined_pred_mask = np.zeros_like(pred_masks[0], dtype=bool)
    filtered_pred_masks = np.zeros_like(pred_masks, dtype=bool)
      
    for gt_mask in gt_masks:
        combined_gt_mask = np.logical_or(combined_gt_mask, gt_mask)

    for pred_mask in pred_masks:
        combined_pred_mask = np.logical_or(combined_pred_mask, pred_mask.astype(bool))

    intersection = np.sum(np.logical_and(combined_gt_mask, combined_pred_mask))
    union = np.sum(np.logical_or(combined_gt_mask, combined_pred_mask))

    iou = intersection/union
    
    true_positive_mask = np.logical_and(combined_gt_mask, combined_pred_mask)
    false_negative_mask = np.logical_and(combined_gt_mask, np.logical_not(combined_pred_mask))
    false_positive_mask = np.logical_and(combined_pred_mask, np.logical_not(combined_gt_mask))

    return true_positive_mask, false_positive_mask, false_negative_mask, iou

In [ ]:
import random
import matplotlib.patches as mpatches
from matplotlib import style

random_indices = random.sample(range(len(pred_images)), 8)
# random_indices = [247, 77,232, 111,     66, 204, 64, 79] #111, 133, 205
# random_indices = [140, 106, 136, 44, 197, 152, 7, 46]
# 140 106 136 44
#   7 
images = [pred_images[i] for i in random_indices]
selected_gts = [valid_gts[i] for i in random_indices]
selected_preds = [valid_preds[i] for i in random_indices]
selected_ious = [valid_ious_pred_vs_gt[i] for i in random_indices]

print("Indices:", random_indices, "Images:", images)

fig, axes = plt.subplots(2, 4, figsize=(36, 18))  # Only 2 rows, 4 columns

for j, (image_name, gt_mask, pred_mask, selected_iou) in enumerate(zip(images, selected_gts, selected_preds, selected_ious)):
    row, col = divmod(j, 4)
    image_id = [img['id'] for img in valid_coco_data['images'] if img['file_name'].startswith(image_name.split(".")[0])][0]
    dataset_images = valid_coco_data['images']
    image_filename = [dataset_images[i]['file_name'] for i in range(len(dataset_images)) if dataset_images[i]['id']==image_id][0]
    gt_masks = np.array([dataset_utils.create_mask(valid_coco_data['annotations'][i]['segmentation'][0], (512, 512)) \
                         for i in range(len(valid_coco_data['annotations']))\
                      if valid_coco_data['annotations'][i]['image_id']==image_id])
    
    left_limit = np.mean(selected_iou) - 0.1
    right_limit = np.mean(selected_iou) + 0.05
    thresholds = [[0.6, 0.7], [0.8, 0.9]]

    gt_masks = np.array(gt_masks)
    gt_masks = gt_masks[:, None, :, :]

    image = cv2.imread(valid_dir + image_name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # _, _, _, iou = compute_metrics_with_range(gt_masks, pred_mask, [0.6, 0.7], image)
    # if 0.6 < iou < 0.7 or 0.8 < iou < 0.9:
    #     print(j,  iou)
    # continue
    
    # Display the original image
    ax_simple = axes[row, col]
    ax_simple.imshow(255-image)
    ax_simple.set_xticks([])
    ax_simple.set_yticks([])
    ax_simple.set_title(f'{image_name.split(".")[0].replace("_png", "")}', fontsize=45, pad=10)

    for spine in ax_simple.spines.values():
        spine.set_linewidth(0)
        spine.set_edgecolor('white')

    # Display the true positive, false positive, and false negative masks
    tp_color = np.array([0/255, 255/255, 0/255, 0.4])
    fp_color = np.array([255/255, 0/255, 0/255, 0.4])
    fn_color = np.array([0/255, 0/255, 255/255, 0.4])
    
    for i, threshold in enumerate(thresholds):
        true_positive_mask, false_positive_mask, false_negative_mask, iou = compute_metrics_with_range(gt_masks, pred_mask, threshold, image)
        print(iou)
        if j<4: 
            left_limit = 0.6
            right_limit = 0.7
        else: 
            left_limit = 0.8
            right_limit = 0.9
            
        # if left_limit < iou and iou < right_limit:
        h, w = true_positive_mask.shape[-2:]
        true_positive_mask = true_positive_mask.reshape(h, w, 1) * tp_color.reshape(1, 1, -1)
        false_positive_mask = false_positive_mask.reshape(h, w, 1) * fp_color.reshape(1, 1, -1)
        false_negative_mask = false_negative_mask.reshape(h, w, 1) * fn_color.reshape(1, 1, -1)
        ax_simple.imshow(false_positive_mask)
        ax_simple.imshow(false_negative_mask)
        ax_simple.imshow(true_positive_mask)
        ax_simple.text(20, 480, f'IoU ~ {iou:.2f}', fontsize=30, color='white', bbox=dict(facecolor='black', alpha=0.8))

        if j==0 or j==4:
            ax_simple.set_ylabel(f'{np.round(left_limit, 2)} < IoU < {np.round(right_limit, 2)}', fontsize=45)

print("\nPlotting...")
tp_patch = mpatches.Patch(color=tp_color, label='True Positive', alpha=0.6)
fp_patch = mpatches.Patch(color=fp_color, label='False Positive', alpha=0.6)
fn_patch = mpatches.Patch(color=fn_color, label='False Negative', alpha=0.6)

fig.legend(handles=[tp_patch, fp_patch, fn_patch], loc='lower center', bbox_to_anchor=(0.5, -0.003), ncol=3, fontsize=30)
plt.tight_layout(pad=0.1)
plt.subplots_adjust(left=0.1, right=0.9, top=0.99, bottom=0.05, hspace=0.01, wspace=0.01)
plt.savefig('../plots/iou_tp_fp_fn_images.png', bbox_inches='tight', dpi=100)
plt.show()

In [ ]:
# import random
# import matplotlib.patches as mpatches
# from matplotlib import style

# random_indices = random.sample(range(len(pred_images)), 5)
# random_indices = [45, 75, 141, 116, 87] # 6,
# images = [pred_images[i] for i in random_indices]
# selected_gts = [valid_gts[i] for i in random_indices]
# selected_preds = [valid_preds[i] for i in random_indices]
# selected_ious = [valid_all_ious_pred_vs_gt[i] for i in random_indices]

# print("Indices:", random_indices, "Images:", images)
# mean_s = r'$\bar{x}$' 

# fig, axes = plt.subplots(nrows=3, ncols=len(images), figsize=(30, 15))

# for j, (image_name, gt_mask, pred_mask, selected_iou) in enumerate(zip(images, selected_gts, selected_preds, selected_ious)):

#     image_id = [img['id'] for img in valid_coco_data['images'] if img['file_name'].startswith(image_name.split(".")[0])][0]
#     dataset_images = valid_coco_data['images']
#     image_filename = [dataset_images[i]['file_name'] for i in range(len(dataset_images)) if dataset_images[i]['id']==image_id][0]
#     gt_masks = np.array([dataset_utils.create_mask(valid_coco_data['annotations'][i]['segmentation'][0], (512, 512)) \
#                          for i in range(len(valid_coco_data['annotations']))\
#                       if valid_coco_data['annotations'][i]['image_id']==image_id])

#     print('Mean IoU:', np.mean(selected_iou), 'std:', np.std(selected_iou))
#     gt_masks = np.array(gt_masks)
#     gt_masks = gt_masks[:, None, :, :]
#     ax_simple = axes[0, j]
#     image = cv2.imread(valid_dir + image_name)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert image to RGB
#     ax_simple.imshow(255-image)
#     ax_simple.set_xticks([])
#     ax_simple.set_yticks([])
#     ax_simple.set_title(f'{image_name.split(".")[0].replace("_png", "")}', fontsize=30, pad=10)
#     for spine in ax_simple.spines.values():
#             spine.set_linewidth(0)
#             spine.set_edgecolor('white')
        
#     # Display the image in all rows
#     for row in range(2):
#         ax = axes[row+1, j]
#         ax.imshow(255-image)
#         ax.set_xticks([])
#         ax.set_yticks([])

#         for spine in ax.spines.values():
#             spine.set_linewidth(0)
#             spine.set_edgecolor('white')

#     tp_color = np.array([0/255, 255/255, 0/255, 0.4]) 
#     fp_color = np.array([255/255, 0/255, 0/255, 0.4]) 
#     fn_color = np.array([0/255, 0/255, 255/255, 0.4]) 
        
#     thresholds = [0.3, 0.8] 
#     for i, threshold in enumerate(thresholds):
#         ax = axes[i+1, j]
#         true_positive_mask, false_positive_mask, false_negative_mask = predictor_utils.compute_metrics(
#             gt_masks, 
#             pred_mask, 
#             threshold, 
#             image)

#         h, w = true_positive_mask.shape[-2:]
#         true_positive_mask = true_positive_mask.reshape(h, w, 1) * tp_color.reshape(1, 1, -1)
#         false_positive_mask = false_positive_mask.reshape(h, w, 1) * fp_color.reshape(1, 1, -1)
#         false_negative_mask = false_negative_mask.reshape(h, w, 1) * fn_color.reshape(1, 1, -1)

#         # Overlay the TP, FP, FN masks with respective colors
#         ax.imshow(false_positive_mask)
#         ax.imshow(false_negative_mask)
#         ax.imshow(true_positive_mask)
#         if j == 0:
#             ax.set_ylabel(f'IoU > {threshold}', fontsize=36)
            
# print("\nPlotting...")
# tp_patch = mpatches.Patch(color=tp_color, label='True Positive', alpha=0.8)
# fp_patch = mpatches.Patch(color=fp_color, label='False Positive', alpha=0.8)
# fn_patch = mpatches.Patch(color=fn_color, label='False Negative', alpha=0.8)

# # Add a legend for TP, FP, FN
# fig.legend(handles=[tp_patch, fp_patch, fn_patch], loc='lower center', bbox_to_anchor=(0.5, -0.05), ncol=3, fontsize=30)
# plt.tight_layout(pad=0.1)
# plt.subplots_adjust(left=0.1, right=0.9, top=0.99, bottom=0.05, hspace=0.01, wspace=0.01)
# plt.savefig('../plots/iou_tp_fp_fn_images.png', bbox_inches='tight', dpi=400)
# plt.show()